Criando catalogo para arquitetura medalhao, os schemas(bronze e silver) 
e volume (landing)

In [0]:
%sql 
CREATE CATALOG IF NOT EXISTS medalhao


In [0]:
%sql
USE CATALOG medalhao;
    
CREATE SCHEMA IF NOT EXISTS bronze;
CREATE SCHEMA IF NOT EXISTS silver;

In [0]:
%sql
USE CATALOG medalhao;
USE SCHEMA default;
CREATE VOLUME IF NOT EXISTS landing 

Depois de criar o volume é feito o upload dos csv no mesmo


In [0]:
catalogo = "medalhao"
bronze_db_name = "bronze"

In [0]:
from pyspark.sql import functions as F

# Funcao para ingestao do csv 
def ingest_csv(nome_arquivo, nome_tabela):
    try:
        # Lendo o arquivo csv a partir do seu caminho localizado no volume
        landing_path = f"/Volumes/medalhao/default/landing/{nome_arquivo}"
        df = spark.read.csv(landing_path, header=True, inferSchema=True)

        # Adicionando a coluna de data de ingestao
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())

        # Escrita no formato Delta
        table_full_name = f"{catalogo}.{bronze_db_name}.{nome_tabela}"
        df_with_metadata.write.format("delta").mode("overwrite").saveAsTable(table_full_name)

        print(f"✅ Tabela bronze.{nome_tabela} criada com sucesso!\n")

    except Exception as e:
        print(f"❌ Erro ao criar tabela bronze.{nome_tabela}: {e}")


In [0]:
import requests
import json
from datetime import datetime

# Funcao para ingestao da API do Banco Central
def ingest_cotacao_dolar(data_inicio, data_fim):

    table_full_name = f"{catalogo}.{bronze_db_name}.dm_cotacao_dolar"

    try:
        # Construindo a URL da API
        # Formato esperado da data: MM-DD-AAAA
        url = (
            "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo("
            f"dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?"
            f"@dataInicial='{data_inicio}'&@dataFinalCotacao='{data_fim}'&"
            "$select=dataHoraCotacao,cotacaoCompra&$format=json"
        )
        
        # 2. Fazendo a requisicao HTTP
        print(f"Chamando API: {url}")
        response = requests.get(url)
        response.raise_for_status()

        # Extraindo os dados encontrados em JSON
        data = response.json().get('value', [])
        
        if not data:
            print(f"⚠️ Aviso: Nenhuma cotação de dólar encontrada para o período {data_inicio} a {data_fim}.\n")
            return

        # Convertendo os dados para Spark DataFrame
        df = spark.createDataFrame(data)

        # Adicionando metadados e salvando no formato Delta
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())
        df_with_metadata.write.format("delta").mode("overwrite").saveAsTable(table_full_name)

        print(f"✅ Tabela API {table_full_name} criada com sucesso!")

    except Exception as e:
        print(f"❌ Erro ao criar tabela {table_full_name} (da API): {e}\n")

In [0]:
data_inicio_formatada = "01-01-2016" 
data_fim_formatada = "10-31-2020" 

ingest_cotacao_dolar(data_inicio_formatada, data_fim_formatada)

print("Ingestão da camada Bronze para API do banco central concluída!")

In [0]:
#Testando para vizualizar se os dados foram recebidos corretamente
df2 = spark.table("medalhao.bronze.dm_cotacao_dolar")

display(df2.limit(5))

In [0]:
# Mapeamento do arquivo CSV de origem para o nome da Tabela Delta (Camada Bronze)
ingest_csv("olist_customers_dataset.csv", "ft_consumidores")
ingest_csv("olist_geolocation_dataset.csv", "ft_geolocalizacao")
ingest_csv("olist_order_items_dataset.csv", "ft_itens_pedidos")
ingest_csv("olist_order_payments_dataset.csv", "ft_pagamentos_pedidos")
ingest_csv("olist_order_reviews_dataset.csv", "ft_avaliacoes_pedidos")
ingest_csv("olist_orders_dataset.csv", "ft_pedidos")
ingest_csv("olist_products_dataset.csv", "ft_produtos")
ingest_csv("olist_sellers_dataset.csv", "ft_vendedores")
ingest_csv("product_category_name_translation.csv", "dm_categoria_produtos_traducao")

print("Ingestão da camada Bronze concluída!")